In [3]:
import numpy as np
from typing import List

import cv2
import networkx as nx
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt


class Node:
    """ Describes a node in a graph, and the edges connected
        to that node."""

    def __init__(self, ID, neighbors, descriptor, truth=None, file_path=None):
        """
        Parameters
        ----------
        ID : int
            A unique identifier for this node. Should be a
            value in [0, N-1], if there are N nodes in total.

        neighbors : Sequence[int]
            The node-IDs of the neighbors of this node.

        descriptor : numpy.ndarray
            The shape-(512,) descriptor vector for the face that this node corresponds to.

        truth : Optional[str]
            If you have truth data, for checking your clustering algorithm,
            you can include the label to check your clusters at the end.
            If this node corresponds to a picture of Ryan, this truth
            value can just be "Ryan"

        file_path : Optional[str]
            The file path of the image corresponding to this node, so
            that you can sort the photos after you run your clustering
            algorithm
        """
        self.id = ID  # a unique identified for this node - this should never change

        # The node's label is initialized with the node's ID value at first,
        # this label is then updated during the whispers algorithm
        self.label = ID

        # (n1_ID, n2_ID, ...)
        # The IDs of this nodes neighbors. Empty if no neighbors
        self.neighbors = tuple(neighbors)
        self.descriptor = descriptor

        self.truth = truth
        self.file_path = file_path


def plot_graph(graph, adj):
    """ Use the package networkx to produce a diagrammatic plot of the graph, with
    the nodes in the graph colored according to their current labels.
    Note that only 20 unique colors are available for the current color map,
    so common colors across nodes may be coincidental.
    Parameters
    ----------
    graph : Tuple[Node, ...]
        The graph to plot. This is simple a tuple of the nodes in the graph.
        Each element should be an instance of the `Node`-class.

    adj : numpy.ndarray, shape=(N, N)
        The adjacency-matrix for the graph. Nonzero entries indicate
        the presence of edges.

    Returns
    -------
    Tuple[matplotlib.fig.Fig, matplotlib.axis.Axes]
        The figure and axes for the plot."""

    g = nx.Graph()
    for n, node in enumerate(graph):
        g.add_node(n)

    # construct a network-x graph from the adjacency matrix: a non-zero entry at adj[i, j]
    # indicates that an egde is present between Node-i and Node-j. Because the edges are
    # undirected, the adjacency matrix must be symmetric, thus we only look ate the triangular
    # upper-half of the entries to avoid adding redundant nodes/edges
    g.add_edges_from(zip(*np.where(np.triu(adj) > 0)))

    # we want to visualize our graph of nodes and edges; to give the graph a spatial representation,
    # we treat each node as a point in 2D space, and edges like compressed springs. We simulate
    # all of these springs decompressing (relaxing) to naturally space out the nodes of the graph
    # this will hopefully give us a sensible (x, y) for each node, so that our graph is given
    # a reasonable visual depiction
    pos = nx.spring_layout(g)

    # make a mapping that maps: node-lab -> color, for each unique label in the graph
    color = list(iter(cm.tab20b(np.linspace(0, 1, len(set(i.label for i in graph))))))
    color_map = dict(zip(sorted(set(i.label for i in graph)), color))
    colors = [color_map[i.label] for i in graph]  # the color for each node in the graph, according to the node's label

    # render the visualization of the graph, with the nodes colored based on their labels!
    fig, ax = plt.subplots()
    nx.draw_networkx_nodes(g, pos=pos, ax=ax, nodelist=range(len(graph)), node_color=colors)
    nx.draw_networkx_edges(g, pos, ax=ax, edgelist=g.edges())
    return fig, ax

def whispers_algorithm(adj_matrix: np.ndarray, nodes: List[Node], max_iters=1):
    '''
    adj_matrix: an adjacency matrix that consists of all images (N x N). This is
    used for the weighted sum.
    
    nodes: a list of Node objects. This is used to determine if a node has been visited
    and also allows us to change the label of a node and access its neighbors.
    '''
    N = len(nodes)
    visited = {}
    for _ in range(max_iters):
        while len(visited) < N:
            i = np.random.randint(0, N, size=1)[0]
            node = nodes[i]
            if node.id not in visited:
                weighted_sum_list = {}
                for i in node.neighbors:
                    neighbor = nodes[i]
                    if neighbor.label not in weighted_sum_list:
                        weighted_sum_list[neighbor.label] = adj_matrix[node.id][neighbor.id]
                    else:
                        weighted_sum_list[neighbor.label] += adj_matrix[node.id][neighbor.id]
                if weighted_sum_list:
                    max_weighted_sum = weighted_sum_list[max(weighted_sum_list, key=weighted_sum_list.get)]
                    potential_neighbors = [k for k,v in weighted_sum_list.items() if v == max_weighted_sum]
                    if potential_neighbors:
                        r = np.random.randint(0, len(potential_neighbors), size=1)[0]
                        label = nodes[potential_neighbors[r]].label

                        node.label = label
                visited[node.id] = True

def assess_success(nodes):
    correct_match = 0
    label_match_only = 0
    truth_match_only = 0
    
    for node_i in range(len(nodes)):
        if node_i + 1 < len(nodes):
            if nodes[node_i].label ==nodes[node_i + 1].label and nodes[node_i].truth == nodes[node_i + 1].truth: 
                correct_match += 1
            elif not nodes[node_i].label == nodes[node_i + 1].label and nodes[node_i].truth == nodes[node_i + 1].truth:
                truth_match_only += 1
            elif nodes[node_i].label == nodes[node_i + 1].label and not nodes[node_i].truth == nodes[node_i + 1].truth:
                label_match_only += 1
            else: 
                break
    
    pairwise_precision = correct_match / (correct_match + truth_match_only)
    pairwise_recall = correct_match / (correct_match + label_match_only)
    return pairwise_precision, pairwise_recall

def create_graph(descriptors_list, threshold):
    N = len(descriptors_list)
    nodes = []
    adj_matrix = np.zeros(shape=(N, N))

    for i in range(N):
        for k in range(i+1, N):
            dist = cos_distance(descriptors_list[i], descriptors_list[k])
            if dist < threshold:
                adj_matrix[i][k] = 1 / (dist**2)
                adj_matrix[k][i] = 1 / (dist**2)

    for i in range(N):
        neighbors = []
        for k in range(N):
            if adj_matrix[i][k] > 0:
                neighbors.append(k)
        truth = None
        if i >= 0 and i <= 10:
            truth = 'JB'
        elif i >= 11 and i <= 15:
            truth = 'TS'
        elif i >= 16 and i <= 20:
            truth = 'BS'
        u = Node(i, neighbors, descriptors_list[i], truth)
        nodes.append(u)
    return (adj_matrix, nodes)

adj_matrix, nodes = create_graph(descriptors_list, 0.6)
whispers_algorithm(adj_matrix, nodes, 100)
pairwise_precision, pairwise_recall = assess_success(nodes)
print('precision: {}; recall: {}'.format(pairwise_precision, pairwise_recall))

ZeroDivisionError: division by zero

In [4]:
import os

descriptors_list = []

for directory in os.listdir('database2'):
    directory_to_read = 'database2/' + directory
    for file in os.listdir(directory_to_read):
        print(directory_to_read + '/' + file)
        pic = cv2.imread(directory_to_read + '/' + file)[:,:,::-1]
        boxes, probabilities, _ = model.detect(pic)
        box = [boxes[np.argmax(probabilities)]]
        descriptor = model.compute_descriptors(pic, box) # Producing the descriptor vector
        descriptor = descriptor.reshape(512)
        descriptors_list.append(descriptor)
# np.save('descriptors.npy', np.array(descriptors_list))

cos_distances = []

for i in range(0, len(descriptors_list)):
    for k in range(i+1, len(descriptors_list)):
        dist = cos_distance(descriptors_list[i], descriptors_list[k])
        cos_distances.append(dist)

classes_labels = np.arange(0, 1.5, 0.1)

fig, ax = plt.subplots()
ax.hist(cos_distances, bins=classes_labels)

database2/Shohei Ohtani/8.jpg


NameError: name 'model' is not defined